In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
import re
import unicodedata
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [2]:
def get_url(tot_pages):
    for page in range(1,tot_pages+1):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')       
        
        url_main = f'https://msk.fitnesslook.ru/trenajery/begovye-dorojki/?page={page}//'
        response = requests.get(url_main, headers = {'user-agent': ua.random}, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = 'p_n notbm')
        for item in data:
            url = 'https://msk.fitnesslook.ru' + item['href']
            yield url

In [3]:
def get_item(tot_pages):

    for url_item in get_url(tot_pages):
        response = requests.get(url_item, headers = {'user-agent': ua.random}, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')

        try:
            title = soup.find('h1').text
            title = title.replace('\r', '').replace('\t', '').replace('\n', '').strip()
        except:
            title = ''

        article = ''

        try:
            image = soup.find(class_ = 'a magnifier-thumb-wrapper')
            url = 'https://msk.fitnesslook.ru' + image['href']

        except:
            print(f'Log: no image for {article} found...')
            url = ''

        try:
            char = soup.find(class_ = 'product-options')
            char_lst = []
            for ch in char.children:
                char_lst.append(re.sub(re.compile('\?(.*?)\:'), '', ch.text).strip())
            chars = ' '.join(char_lst)
            chars_fin = chars.split('Преимущества')[0].strip()
        except:
            chars_fin = ''



        desc = ''

        try:
            price = soup.find(class_ = 'currentProductPrice').text
            price = price.replace(' ', '')
        except:
            price =''

        try:
            cat = soup.find(class_ = 'breadcrumbs').text.split('\n\n')
            
        except:
            cat = []
        try:
            cat_1 = cat[-3].replace('\n', '').strip()
        except:
            cat_1 = ''
        try:
            cat_2 = cat[-2].replace('\n', '').strip()
        except:
            cat_2 = ''
        try:
            cat_3 = cat[-1].replace('\n', '').strip()
        except:
            cat_3 = ''

        yield  title, article, price, cat_1, cat_2, cat_3, url, desc, chars_fin

In [4]:
df_columns = ['title', 'article','price', 'cat_1', 'cat_2', 'cat_3','url', 
              'description', 'caracteristics']

df = pd.DataFrame(columns = df_columns)
save_frequency = 300
log_frequency = 10

In [5]:
def to_csv(tot_pages):
    counter = 0  

    for item in get_item(tot_pages):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
            #time.sleep(0.05)

            counter += 1
            if counter % save_frequency == 0:
                df.to_csv('fitnesslook.csv')
                print(f'{counter} items saved in csv..')
    df.to_csv('fitnesslook.csv')

    print(f'\nTotal items saved: {counter}')  

In [6]:
to_csv(32)

Log: working over page 10...
300 items saved in csv..
Log: working over page 20...
600 items saved in csv..
Log: working over page 30...
900 items saved in csv..

Total items saved: 954


In [7]:
df

,title,article,price,cat_1,cat_2,cat_3,url,description,caracteristics
0,Беговая дорожка Orlauf Shrike в Москве,,31900,Кардиотренажеры,Беговые дорожки,Orlauf Shrike,https://msk.fitnesslook.ru/i/product_i/12710_1...,,Характеристики Shrike Тип дорожки электрическа...
1,Беговая дорожка Orlauf Stellar в Москве,,52400,Кардиотренажеры,Беговые дорожки,Orlauf Stellar,https://msk.fitnesslook.ru/i/product_i/15502_3...,,Характеристики Stellar Тип дорожки электрическ...
2,Беговая дорожка Oxygen New Classic Aurum TFT в...,,142990,Кардиотренажеры,Беговые дорожки,Oxygen New Classic Aurum TFT,https://msk.fitnesslook.ru/i/product_i/10001_1...,,Характеристики New Classic Aurum TFT Тип дорож...
3,Беговая дорожка Oxygen M-Concept Sport Black в...,,104990,Кардиотренажеры,Беговые дорожки,Oxygen M-Concept Sport Black,https://msk.fitnesslook.ru/i/product_i/13845_6...,,Характеристики M-Concept Sport Black Тип дорож...
4,Беговая дорожка Orlauf Merlin A в Москве,,59900,Кардиотренажеры,Беговые дорожки,Orlauf Merlin A,https://msk.fitnesslook.ru/i/product_i/15984_1...,,Характеристики Merlin A Тип дорожки электричес...
...,...,...,...,...,...,...,...,...,...
949,Беговая дорожка Koenigsmann MS1.0 в Москве,,67990,Кардиотренажеры,Беговые дорожки,Koenigsmann MS1.0,https://msk.fitnesslook.ru/i/product_i/16251_1...,,Характеристики MS1.0 Тип дорожки механическая ...
950,Беговая дорожка Koenigsmann E1.0 в Москве,,99990,Кардиотренажеры,Беговые дорожки,Koenigsmann E1.0,https://msk.fitnesslook.ru/i/product_i/16261_5...,,Характеристики E1.0 Тип дорожки электрическая ...
951,Беговая дорожка Koenigsmann S1.0 в Москве,,142990,Кардиотренажеры,Беговые дорожки,Koenigsmann S1.0,https://msk.fitnesslook.ru/i/product_i/16262_8...,,Характеристики S1.0 Тип дорожки электрическая ...
952,Беговая дорожка Koenigsmann XR3 в Москве,,200990,Кардиотренажеры,Беговые дорожки,Koenigsmann XR3,https://msk.fitnesslook.ru/i/product_i/16264_4...,,Тип дорожки электрическая Использование профес...
